In [14]:
#testing out opening a file
import re

with open('ascl_github_repos') as fp:
    for line in fp:
        match = re.match("^.*github.com/(.*)/(.*)$", line)
        if match:
            author = match.group(1)
            repo = match.group(2)
            #print("author: " + author + " repo: " + repo)

In [1]:
#testing out a GET request to the GitHub API and adding it to a DataFrame
import requests
import github_config
import numpy as np
import pandas as pd

df = pd.DataFrame(columns=['Repo', 'Author', 'Language', 'Bytes'])

session = requests.Session()
session.auth = (github_config.username, github_config.password)

http_base = 'https://api.github.com/repos/'
r = session.get(http_base+'0satoken/Eclairs'+'/languages')
print(r)
json = r.json()

print(json)

for key in json:
    df.loc[0 if pd.isnull(df.index.max()) else df.index.max() + 1] = ['Eclairs', '0satoken', key, json[key]]
    
    '''
    df2 = pd.DataFrame({
        'Repo':'Eclairs',
        'Author':'0satoken',
        'Language':i,
        'Bytes':[json[i]]
    })
    df = df.append(df2)
    '''
print("\n\n")
print(df)

<Response [200]>
{'C++': 219978, 'Jupyter Notebook': 46397, 'Makefile': 2142}



      Repo    Author          Language   Bytes
0  Eclairs  0satoken               C++  219978
1  Eclairs  0satoken  Jupyter Notebook   46397
2  Eclairs  0satoken          Makefile    2142


In [4]:
#The real deal
import re
import requests
import github_config
import numpy as np
import pandas as pd

session = requests.Session()
session.auth = (github_config.username, github_config.password)

http_base = 'https://api.github.com/repos/'

#placeholder column names
df = pd.DataFrame(columns=['Repo', 'Author', 'Language', 'Bytes'])

with open('ascl_github_repos') as fp:
    for line in fp:
        #if it matches the github repo URL scheme 
        # (weeds out malformed duplicates)
        match = re.match("^.*github.com/(.*)/(.*)$", line)
        if match:
            author = match.group(1)
            repo = match.group(2)
            
            #hit the GitHub API
            r = session.get(http_base+author+'/'+repo+'/languages')
            languages = r.json()
            
            for key in languages:
                #adds each language to the end of the dataframe
                df.loc[0 if pd.isnull(df.index.max()) else df.index.max() + 1] = [repo, author, key, languages[key]]

                
print(df)

                     Repo       Author          Language     Bytes
0                 Eclairs     0satoken               C++    219978
1                 Eclairs     0satoken  Jupyter Notebook     46397
2                 Eclairs     0satoken          Makefile      2142
3     Crab.Toolkit.michi2         1054            Python   1199268
4     Crab.Toolkit.michi2         1054             Shell    862626
5     Crab.Toolkit.michi2         1054           Fortran    458353
6     Crab.Toolkit.michi2         1054            Prolog    173935
7     Crab.Toolkit.michi2         1054               IDL    107971
8     Crab.Toolkit.michi2         1054          Makefile      2800
9                   PRISM        1313e            Python    910299
10                  PRISM        1313e  Jupyter Notebook     78845
11                  PRISM        1313e               TeX      6351
12             exostriker   3fon3fonov            Python   5826713
13             exostriker   3fon3fonov           Fortran   174

In [5]:
df.to_csv('languages.csv')